## 特征优化

### 导入数据

In [1]:
import pandas as pd

train_data_file = "./data/zhengqi_train.txt"
test_data_file =  "./data/zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: './zhengqi_train.txt'

### 定义特征构造方法，构造特征

In [3]:
epsilon=1e-5

#组交叉特征，可以自行定义，如增加： x*x/y, log(x)/y 等等。
func_dict = {
            'add': lambda x,y: x+y,
            'mins': lambda x,y: x-y,
            'div': lambda x,y: x/(y+epsilon),
            'multi': lambda x,y: x*y
            }

### 定义特征构造的函数

In [4]:
def auto_features_make(train_data,test_data,func_dict,col_list):
    train_data, test_data = train_data.copy(), test_data.copy()
    for col_i in col_list:
        for col_j in col_list:
            for func_name, func in func_dict.items():
                for data in [train_data,test_data]:
                    func_features = func(data[col_i],data[col_j])
                    col_func_features = '-'.join([col_i,func_name,col_j])
                    data[col_func_features] = func_features
    return train_data,test_data

### 对训练集和测试集数据进行特征构造

In [5]:
train_data2, test_data2 = auto_features_make(train_data,test_data,func_dict,col_list=test_data.columns)

In [6]:
from sklearn.decomposition import PCA   #主成分分析法

#PCA方法降维
pca = PCA(n_components=500)
train_data2_pca = pca.fit_transform(train_data2.iloc[:,0:-1])
test_data2_pca = pca.transform(test_data2)
train_data2_pca = pd.DataFrame(train_data2_pca)
test_data2_pca = pd.DataFrame(test_data2_pca)
train_data2_pca['target'] = train_data2['target']

In [7]:
X_train2 = train_data2[test_data2.columns].values
y_train = train_data2['target']

### 使用lightgbm模型对新构造的特征进行模型训练和评估

In [ ]:
# ls_validation i
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

# 5折交叉验证
Folds=5
kf = KFold(len(X_train2), n_splits=Folds, random_state=2019, shuffle=True)
# 记录训练和预测MSE
MSE_DICT = {
    'train_mse':[],
    'test_mse':[]
}

# 线下训练预测
for i, (train_index, test_index) in enumerate(kf.split(X_train2)):
    # lgb树模型
    lgb_reg = lgb.LGBMRegressor(
        learning_rate=0.01,
        max_depth=-1,
        n_estimators=5000,
        boosting_type='gbdt',
        random_state=2019,
        objective='regression',
    )
   
    # 切分训练集和预测集
    X_train_KFold, X_test_KFold = X_train2[train_index], X_train2[test_index]
    y_train_KFold, y_test_KFold = y_train[train_index], y_train[test_index]
    
    # 训练模型
    lgb_reg.fit(
            X=X_train_KFold,y=y_train_KFold,
            eval_set=[(X_train_KFold, y_train_KFold),(X_test_KFold, y_test_KFold)],
            eval_names=['Train','Test'],
            early_stopping_rounds=100,
            eval_metric='MSE',
            verbose=50
        )


    # 训练集预测 测试集预测
    y_train_KFold_predict = lgb_reg.predict(X_train_KFold,num_iteration=lgb_reg.best_iteration_)
    y_test_KFold_predict = lgb_reg.predict(X_test_KFold,num_iteration=lgb_reg.best_iteration_) 
    
    print('第{}折 训练和预测 训练MSE 预测MSE'.format(i))
    train_mse = mean_squared_error(y_train_KFold_predict, y_train_KFold)
    print('------\n', '训练MSE\n', train_mse, '\n------')
    test_mse = mean_squared_error(y_test_KFold_predict, y_test_KFold)
    print('------\n', '预测MSE\n', test_mse, '\n------\n')
    
    MSE_DICT['train_mse'].append(train_mse)
    MSE_DICT['test_mse'].append(test_mse)
print('------\n', '训练MSE\n', MSE_DICT['train_mse'], '\n', np.mean(MSE_DICT['train_mse']), '\n------')
print('------\n', '预测MSE\n', MSE_DICT['test_mse'], '\n', np.mean(MSE_DICT['test_mse']), '\n------')

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418978	Test's l2: 0.106374
[100]	Train's l2: 0.203693	Test's l2: 0.02276
[150]	Train's l2: 0.114486	Test's l2: 0.00527795
[200]	Train's l2: 0.0741934	Test's l2: 5.99266e-05
[250]	Train's l2: 0.0535396	Test's l2: 0.00036171
[300]	Train's l2: 0.041529	Test's l2: 0.00267813
Early stopping, best iteration is:
[221]	Train's l2: 0.0640274	Test's l2: 6.95547e-08
第0折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.0640273654375399 
------
------
 预测MSE
 6.954770450692039e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419128	Test's l2: 0.142103
[100]	Train's l2: 0.203838	Test's l2: 0.0997735
[150]	Train's l2: 0.114537	Test's l2: 0.0765469
[200]	Train's l2: 0.0742645	Test's l2: 0.0612271
[250]	Train's l2: 0.0536164	Test's l2: 0.0471729
[300]	Train's l2: 0.0415117	Test's l2: 0.0427081
[350]	Train's l2: 0.0333767	Test's l2: 0.0399094
[400]	Train's l2: 0.0275956	Test's l2: 0.0375303
[4

[150]	Train's l2: 0.114559	Test's l2: 0.0172525
[200]	Train's l2: 0.0742264	Test's l2: 0.00504282
[250]	Train's l2: 0.0535831	Test's l2: 0.00153378
[300]	Train's l2: 0.0414425	Test's l2: 7.48706e-06
[350]	Train's l2: 0.0333508	Test's l2: 0.000496099
[400]	Train's l2: 0.0275969	Test's l2: 0.00111861
Early stopping, best iteration is:
[305]	Train's l2: 0.040494	Test's l2: 3.75824e-08
第12折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.04049397966649528 
------
------
 预测MSE
 3.758870105458577e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419187	Test's l2: 0.016239
[100]	Train's l2: 0.203862	Test's l2: 0.0128048
[150]	Train's l2: 0.114614	Test's l2: 0.0104147
[200]	Train's l2: 0.0743045	Test's l2: 0.00526246
[250]	Train's l2: 0.0536267	Test's l2: 0.00508631
[300]	Train's l2: 0.0415942	Test's l2: 0.00615596
Early stopping, best iteration is:
[219]	Train's l2: 0.0649886	Test's l2: 0.00402294
第13折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.06498858566375795 
-----

[850]	Train's l2: 0.00689111	Test's l2: 0.000434364
[900]	Train's l2: 0.00603225	Test's l2: 0.000305302
[950]	Train's l2: 0.00529106	Test's l2: 0.000337115
[1000]	Train's l2: 0.00465794	Test's l2: 0.000242977
[1050]	Train's l2: 0.00409975	Test's l2: 0.000257165
Early stopping, best iteration is:
[985]	Train's l2: 0.00483596	Test's l2: 0.000207193
第20折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.004835961628789988 
------
------
 预测MSE
 0.00020719455433872638 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419024	Test's l2: 0.00281614
[100]	Train's l2: 0.203791	Test's l2: 0.00726691
Early stopping, best iteration is:
[19]	Train's l2: 0.696433	Test's l2: 0.000226693
第21折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.6964330189928877 
------
------
 预测MSE
 0.00022669317466322212 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.41894	Test's l2: 0.130452
[100]	Train's l2: 0.203652	Test's l2: 0.0556921
[150]	Train's l2: 0.114451	

[550]	Train's l2: 0.0164378	Test's l2: 0.0739694
Early stopping, best iteration is:
[484]	Train's l2: 0.0204115	Test's l2: 0.0700455
第47折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.020411548803305615 
------
------
 预测MSE
 0.07004547029259867 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419116	Test's l2: 0.0139555
[100]	Train's l2: 0.203761	Test's l2: 0.0200994
Early stopping, best iteration is:
[1]	Train's l2: 0.95122	Test's l2: 0.00524442
第48折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.9512195532986107 
------
------
 预测MSE
 0.005244422362982837 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419094	Test's l2: 0.0355316
[100]	Train's l2: 0.203786	Test's l2: 0.00930918
[150]	Train's l2: 0.114585	Test's l2: 0.00445071
[200]	Train's l2: 0.0742982	Test's l2: 0.00220223
[250]	Train's l2: 0.0535865	Test's l2: 0.000789025
[300]	Train's l2: 0.0415061	Test's l2: 0.000862087
[350]	Train's l2: 0.0333907	Test's l2: 0.00059014

[100]	Train's l2: 0.203552	Test's l2: 0.000237345
[150]	Train's l2: 0.114507	Test's l2: 0.0228184
[200]	Train's l2: 0.074243	Test's l2: 0.0570426
Early stopping, best iteration is:
[103]	Train's l2: 0.195809	Test's l2: 1.15968e-08
第60折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.19580910435823828 
------
------
 预测MSE
 1.1595788571758307e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419214	Test's l2: 0.00136948
[100]	Train's l2: 0.203877	Test's l2: 1.15152e-05
[150]	Train's l2: 0.114607	Test's l2: 2.82422e-05
[200]	Train's l2: 0.0743208	Test's l2: 0.000240938
Early stopping, best iteration is:
[123]	Train's l2: 0.153558	Test's l2: 9.43635e-09
第61折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.15355832863548302 
------
------
 预测MSE
 9.4351465622612e-09 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419062	Test's l2: 0.0669472
[100]	Train's l2: 0.203879	Test's l2: 0.0694394
Early stopping, best iteration is:
[1]	Trai

...

第943折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.031187885539130357 
------
------
 预测MSE
 7.631494513831846e-09 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418825	Test's l2: 1.07745
[100]	Train's l2: 0.203614	Test's l2: 0.570346
[150]	Train's l2: 0.11439	Test's l2: 0.332753
[200]	Train's l2: 0.0740018	Test's l2: 0.213333
[250]	Train's l2: 0.0533248	Test's l2: 0.140618
[300]	Train's l2: 0.0412507	Test's l2: 0.0899567
[350]	Train's l2: 0.0332076	Test's l2: 0.0619395
[400]	Train's l2: 0.027354	Test's l2: 0.0452283
[450]	Train's l2: 0.0229527	Test's l2: 0.0332123
[500]	Train's l2: 0.0193246	Test's l2: 0.0260882
[550]	Train's l2: 0.0164174	Test's l2: 0.0229998
[600]	Train's l2: 0.014072	Test's l2: 0.0175933
[650]	Train's l2: 0.0121331	Test's l2: 0.0157548
[700]	Train's l2: 0.0105077	Test's l2: 0.0158231
[750]	Train's l2: 0.00910421	Test's l2: 0.0152835
[800]	Train's l2: 0.00790734	Test's l2: 0.0139079
[850]	Train's l2: 0.00690492	Test's l2: 0.0133383
[

------
 训练MSE
 0.15889257817179744 
------
------
 预测MSE
 0.00022316238033547498 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418517	Test's l2: 1.55611
[100]	Train's l2: 0.203258	Test's l2: 1.25641
[150]	Train's l2: 0.114072	Test's l2: 1.12343
[200]	Train's l2: 0.0736814	Test's l2: 1.04698
[250]	Train's l2: 0.0530827	Test's l2: 0.964975
[300]	Train's l2: 0.0411239	Test's l2: 0.905374
[350]	Train's l2: 0.0330622	Test's l2: 0.886526
[400]	Train's l2: 0.0272975	Test's l2: 0.851491
[450]	Train's l2: 0.022851	Test's l2: 0.845616
[500]	Train's l2: 0.0192256	Test's l2: 0.844041
[550]	Train's l2: 0.0163369	Test's l2: 0.843177
[600]	Train's l2: 0.0140014	Test's l2: 0.843727
[650]	Train's l2: 0.0120442	Test's l2: 0.837529
[700]	Train's l2: 0.0104159	Test's l2: 0.834982
[750]	Train's l2: 0.00903028	Test's l2: 0.823283
[800]	Train's l2: 0.00786578	Test's l2: 0.81191
[850]	Train's l2: 0.00688134	Test's l2: 0.802421
[900]	Train's l2: 0.00604151	Test's l2:

[550]	Train's l2: 0.0163845	Test's l2: 1.65355e-05
Early stopping, best iteration is:
[462]	Train's l2: 0.0220183	Test's l2: 4.39791e-08
第965折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.022018308078639186 
------
------
 预测MSE
 4.3980346858188416e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418992	Test's l2: 0.252815
[100]	Train's l2: 0.203701	Test's l2: 0.136898
[150]	Train's l2: 0.114564	Test's l2: 0.0818144
[200]	Train's l2: 0.0743277	Test's l2: 0.0631448
[250]	Train's l2: 0.0536207	Test's l2: 0.0394998
[300]	Train's l2: 0.0414346	Test's l2: 0.0344406
[350]	Train's l2: 0.033346	Test's l2: 0.034559
[400]	Train's l2: 0.027527	Test's l2: 0.0356315
Early stopping, best iteration is:
[329]	Train's l2: 0.0363987	Test's l2: 0.0313402
第966折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.036398712679786026 
------
------
 预测MSE
 0.031340209373359657 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418806	Test's l2: 0.73108

[150]	Train's l2: 0.114492	Test's l2: 0.337343
[200]	Train's l2: 0.0742548	Test's l2: 0.229358
[250]	Train's l2: 0.0535575	Test's l2: 0.168785
[300]	Train's l2: 0.0414817	Test's l2: 0.142453
[350]	Train's l2: 0.0333824	Test's l2: 0.114718
[400]	Train's l2: 0.0275696	Test's l2: 0.0972589
[450]	Train's l2: 0.0231083	Test's l2: 0.0939547
[500]	Train's l2: 0.0194555	Test's l2: 0.0921853
[550]	Train's l2: 0.0165164	Test's l2: 0.0899976
[600]	Train's l2: 0.0141038	Test's l2: 0.0872106
[650]	Train's l2: 0.0121428	Test's l2: 0.0792903
[700]	Train's l2: 0.0105141	Test's l2: 0.0791937
[750]	Train's l2: 0.00914019	Test's l2: 0.0757362
[800]	Train's l2: 0.00794836	Test's l2: 0.0779471
[850]	Train's l2: 0.00694014	Test's l2: 0.079302
Early stopping, best iteration is:
[757]	Train's l2: 0.00895987	Test's l2: 0.0748623
第977折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.0089598653999946 
------
------
 预测MSE
 0.07486227605812884 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's

[250]	Train's l2: 0.053579	Test's l2: 0.0405772
[300]	Train's l2: 0.0415079	Test's l2: 0.031866
[350]	Train's l2: 0.0334016	Test's l2: 0.0305836
[400]	Train's l2: 0.0275541	Test's l2: 0.0263975
[450]	Train's l2: 0.023084	Test's l2: 0.0231882
[500]	Train's l2: 0.0194329	Test's l2: 0.0220263
[550]	Train's l2: 0.0165213	Test's l2: 0.0196946
[600]	Train's l2: 0.0141338	Test's l2: 0.0186575
[650]	Train's l2: 0.0121927	Test's l2: 0.0186627
Early stopping, best iteration is:
[589]	Train's l2: 0.0146204	Test's l2: 0.0182939
第988折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.014620376910379002 
------
------
 预测MSE
 0.01829390885124208 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418756	Test's l2: 0.709159
[100]	Train's l2: 0.20363	Test's l2: 0.24395
[150]	Train's l2: 0.114466	Test's l2: 0.0750071
[200]	Train's l2: 0.0741926	Test's l2: 0.0218276
[250]	Train's l2: 0.053533	Test's l2: 0.00357041
[300]	Train's l2: 0.0414059	Test's l2: 0.00068012
[350]	Train's l2: